# Where do our current fails ?

Can we see what enables the model to make such accurate predictions ?

As of right now this notebook does not bring any insight unfortunately !

In [1]:
# Unfortunately the following code doesn't work...
'''
import pyarrow as pa
import os

wd = os.path.join(os.getcwd(), '..', '..')
dataset_path = os.path.join(wd, "autotrain-data-histopathological_image_classification/processed/train/dataset.arrow")
print(dataset_path)

# Load the Apache Arrow file
file = pa.ipc.open_file(dataset_path) 

# Read the contents of the file into an Apache Arrow table
table = file.read_all()

# Convert the Apache Arrow table to a Pandas DataFrame
df = table.to_pandas()

# Now you can work with the DataFrame as you would with any other Pandas DataFrame
print(df.head())
'''

'\nimport pyarrow as pa\nimport os\n\nwd = os.path.join(os.getcwd(), \'..\', \'..\')\ndataset_path = os.path.join(wd, "autotrain-data-histopathological_image_classification/processed/train/dataset.arrow")\nprint(dataset_path)\n\n# Load the Apache Arrow file\nfile = pa.ipc.open_file(dataset_path) \n\n# Read the contents of the file into an Apache Arrow table\ntable = file.read_all()\n\n# Convert the Apache Arrow table to a Pandas DataFrame\ndf = table.to_pandas()\n\n# Now you can work with the DataFrame as you would with any other Pandas DataFrame\nprint(df.head())\n'

The solution is now to predict again on the entire dataset and to analyse where mistakes are made and why.

In [2]:
import pandas as pd
import os
from pathlib import Path
import cv2
import mahotas
import numpy as np

In [3]:
wd = os.path.join(os.getcwd(), '..')
data = os.path.join(wd, 'data', 'input')
image_train_path = os.path.join(data, 'Train')
image_test_path = os.path.join(data, 'Test')

csv_output_path = os.path.join(wd, 'data', 'output', 'huggingface_autotrain', 'histopathological_image_classification.csv')
submission_path = os.path.join(wd, 'data', 'output', 'submission', 'pred_swim_20231002.csv')
error_analysis_path = os.path.join(wd, 'data', 'output', 'error_analysis', 'pred_swim_20231002.csv')

# get images list from folder using os.listdir
images = os.listdir(path=image_train_path)
images = [image for image in images if Path(image).suffix == '.png']

In [4]:
df = pd.read_csv(error_analysis_path)
df.head(3)

,procedure,class,type,magnification,slide,mag,seq,filename,type_id_pred,type_id
0,SOB,B,A,14,22549AB,100,1,SOB_B_A-14-22549AB-100-001.png,7,7
1,SOB,B,A,14,22549AB,100,2,SOB_B_A-14-22549AB-100-002.png,7,7
2,SOB,B,A,14,22549AB,100,3,SOB_B_A-14-22549AB-100-003.png,7,7


## Evaluation and metrics

On which class do I fail, and how to measure it ?\
One thing I could do is first look at where the prediction differs from the type, \
then check the volume of error on each type compared to the volume of the type in the train set. \
Another thing would be to check the confusion matrix. (original as rows, predictions as columns)

After that I'll look at some measures available for that matter.

In [5]:
# End table should look like this : 
# type_id, volume, volume of error (error different from it), ratio vol error/vol
df["is_error"] = 0
df.loc[df["type_id_pred"] != df["type_id"], "is_error"] = 1

In [6]:
df[["type_id", "type_id_pred", ]].groupby("type_id").count()

,type_id_pred
type_id,
1,72
2,46
3,92
4,57
5,58
6,16
7,65
8,16


In [7]:
df[["type_id", "is_error", ]].groupby("type_id").sum()

,is_error
type_id,
1,3
2,0
3,1
4,2
5,0
6,3
7,0
8,0


Volume of error is extremely low already !

In [8]:
vol = df[["type_id", "type_id_pred", ]].groupby("type_id").count()
vol_error = df[["type_id", "is_error", ]].groupby("type_id").sum()
error_analysis_naiv = pd.concat([vol, vol_error], axis=1)
error_analysis_naiv["ratio"] = error_analysis_naiv["is_error"]/error_analysis_naiv["type_id_pred"] 
error_analysis_naiv

,type_id_pred,is_error,ratio
type_id,,,
1,72,3,0.041667
2,46,0,0.000000
3,92,1,0.010870
4,57,2,0.035088
5,58,0,0.000000
6,16,3,0.187500
7,65,0,0.000000
8,16,0,0.000000


Clearly type_id 6 is more difficult.
Then type 1, type 4.

In [9]:
df[df["type_id_pred"] != df["type_id"]]

,procedure,class,type,magnification,slide,mag,seq,filename,type_id_pred,type_id,is_error
86,SOB,B,F,14,21998CD,100,1,SOB_B_F-14-21998CD-100-001.png,4,1,1
126,SOB,B,F,14,29960AB,100,5,SOB_B_F-14-29960AB-100-005.png,6,1,1
131,SOB,B,F,14,29960AB,100,10,SOB_B_F-14-29960AB-100-010.png,7,1,1
146,SOB,B,PT,14,21998AB,100,15,SOB_B_PT-14-21998AB-100-015.png,1,4,1
168,SOB,B,PT,14,21998AB,100,45,SOB_B_PT-14-21998AB-100-045.png,3,4,1
265,SOB,M,LC,14,16196,100,4,SOB_M_LC-14-16196-100-004.png,5,6,1
268,SOB,M,LC,14,16196,100,14,SOB_M_LC-14-16196-100-014.png,5,6,1
271,SOB,M,LC,14,16196,100,17,SOB_M_LC-14-16196-100-017.png,5,6,1
397,SOB,M,PC,14,19440,100,16,SOB_M_PC-14-19440-100-016.png,2,3,1


Well, after looking at the images.\
How do I put this.\
Type 6 is a dense harsh pink.\
A simple model on top of the actual one should be enough to correct this.\
However what scares me is that there do not seems to be a lot of LC in the test folder.

In [10]:
# Let's try to compute a simple value on hue (frankly could be just the amount of red in the image)
# using a random forest or xgboost for classification
# as an input I'll use the output of the previous model and features from the tutorial (which look good !)

# parameters for the features extraction
bins = 8

# functions to extract features from the images
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

def fd_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# Compute the features
feat_haralick = []
feat_histogram = []
feat_hu = []

for i in images:
    img = cv2.imread(os.path.join(image_train_path, i))  # Load image
    feat_hu.append(fd_hu_moments(img))  # Append Hu values
    feat_haralick.append(fd_haralick(img))  # Append Haralick values
    feat_histogram.append(fd_histogram(img))  # Append Histogram values
    
# Concatenate with the original data frame 
df_features = np.hstack([feat_haralick, feat_hu, feat_histogram])
df_features = pd.DataFrame(df_features)
df_features.shape

(422, 532)

In [11]:
df_features.head(3)

,0,1,2,3,4,5,6,7,8,9,...,522,523,524,525,526,527,528,529,530,531
0,0.002159,56.342601,0.957225,658.536578,0.285724,368.130680,2577.803709,7.345296,10.541800,0.000437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.001479,56.489012,0.952225,591.148607,0.263477,361.274977,2308.105416,7.320420,10.625002,0.000410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000891,77.383048,0.944581,698.171173,0.236460,355.468091,2715.301645,7.581967,11.145510,0.000348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_features.iloc[:, 20:]

,20,21,22,23,24,25,26,27,28,29,...,522,523,524,525,526,527,528,529,530,531
0,0.0,0.0,0.000000,0.000010,0.000153,0.002980,0.010285,0.000038,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000033,0.000251,0.002619,0.006585,0.000017,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000021,0.000313,0.001817,0.008845,0.000230,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000011,0.000219,0.001317,0.010303,0.019070,0.000121,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000009,0.000112,0.001068,0.004892,0.000034,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.0,0.0,0.000000,0.000314,0.000838,0.001215,0.093242,0.170853,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
418,0.0,0.0,0.000011,0.001015,0.002604,0.003609,0.021818,0.008288,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
419,0.0,0.0,0.000000,0.000000,0.000078,0.000039,0.001679,0.000379,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
420,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000364,0.001717,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model error

With sklearn random forest.\
On such a large dataset (but not long) it might be better to take a somewhat short tree but testing a lot of covariables.  

In [13]:
df_features['type_id_pred_img'] = df['type_id_pred']

In [14]:
df_features

,0,1,2,3,4,5,6,7,8,9,...,523,524,525,526,527,528,529,530,531,type_id_pred_img
0,0.002159,56.342601,0.957225,658.536578,0.285724,368.130680,2577.803709,7.345296,10.541800,0.000437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
1,0.001479,56.489012,0.952225,591.148607,0.263477,361.274977,2308.105416,7.320420,10.625002,0.000410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,0.000891,77.383048,0.944581,698.171173,0.236460,355.468091,2715.301645,7.581967,11.145510,0.000348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
3,0.000831,104.921993,0.937596,840.643381,0.188244,354.103247,3257.651532,7.658306,11.474236,0.000271,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
4,0.001277,68.442074,0.946159,635.620753,0.255009,360.947441,2474.040937,7.403313,10.811924,0.000393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,0.001084,90.199284,0.967643,1393.740553,0.236030,324.997978,5484.762930,7.702850,11.214430,0.000321,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
418,0.000962,130.097530,0.952844,1379.382435,0.226041,328.563393,5387.432208,7.701500,11.378433,0.000315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
419,0.000698,78.016717,0.957054,908.311793,0.201126,303.955233,3555.230455,7.594811,11.258012,0.000291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
420,0.000507,115.440718,0.945563,1060.279872,0.145113,329.516178,4125.678771,7.709068,11.698973,0.000225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

In [42]:
def drop_zero_sum_columns(df):
    cols_to_drop = []
    for col in df.columns:
        if df[col].sum() == 0:
            cols_to_drop.append(col)
    df = df.drop(cols_to_drop, axis=1)
    return df

df_features = drop_zero_sum_columns(df_features)
df_features.head(3)

,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,400,type_id_pred_img
0,0.002159,56.342601,0.957225,658.536578,0.285724,368.130680,2577.803709,7.345296,10.541800,0.000437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
1,0.001479,56.489012,0.952225,591.148607,0.263477,361.274977,2308.105416,7.320420,10.625002,0.000410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,0.000891,77.383048,0.944581,698.171173,0.236460,355.468091,2715.301645,7.581967,11.145510,0.000348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [ ]:
X = df_features
X.columns = X.columns.astype(str)
y = df['type_id']

In [70]:
def replace_values(s):
    s = s.apply(lambda x: 1 if x == 6 else 0)
    return s

y = replace_values(y)

In [104]:
clf = RandomForestClassifier(max_depth=None, random_state=20230213)
cross = cross_validate(clf, X, y, cv=3, return_estimator=True) # stratified by default

In [105]:
cross

{'fit_time': array([0.22752213, 0.2230022 , 0.21999907]),
 'score_time': array([0.01797748, 0.01902771, 0.01950121]),
 'estimator': [RandomForestClassifier(random_state=20230213),
  RandomForestClassifier(random_state=20230213),
  RandomForestClassifier(random_state=20230213)],
 'test_score': array([0.96453901, 0.96453901, 0.87857143])}

In [106]:
clf_val = cross['estimator'][2]

In [107]:
confusion_matrix(y, clf_val.predict(X))

array([[391,  15],
       [  2,  14]], dtype=int64)